In [ ]:

from fa_xla import XLAFlashAttentionWrapper,FLASH_ATTENTION_AVAILABLE
from transformers.models.qwen2.modeling_qwen2 import apply_rotary_pos_emb ## some may have thier own 
model = AutoModelForCausalLM.from_pretrained(MODEL,torch_dtype=torch.bfloat16,trust_remote_code=True)

for layer in model.model.layers:
    xs.apply_backward_optimization_barrier(layer)
    
with torch.no_grad():
    if FLASH_ATTENTION_AVAILABLE:
        if FLAGS['OPTIMIZER'] == 'adamw': 
            print("Adam or any 2nd order optimizer Doesn't Work with FA with GQA ; use Lion or SGD or Use MHA ;")
        
        old_atten_config=[]
        
        mesh = xs.get_global_mesh() #Currently the mesh is (4,1)
        partition_spec = (0, None,None,None) ## Always try to shard along Batch Dimension 
        
        # Replace attention mechanism with XLA Flash Attention
        for layer in model.model.layers:
            original_attention = layer.self_attn
            layer.self_attn = XLAFlashAttentionWrapper(original_attention, mesh, partition_spec,rotary_func=apply_rotary_pos_emb)
            old_atten_config.append(original_attention)
        print("Applied Flash Attention for TPU ; Saved Old Attention config for replacement after training")


In [ ]:
### Train ....

In [ ]:
# Unwrap the XLA Flash Attention
model = model.cpu()
if FLASH_ATTENTION_AVAILABLE :
    try:
        for index,attn in enumerate(old_atten_config):
            model.model.layers[index].self_attn=attn
        print("Replaced XLAFLashAttenWrapper With Original Attention Implementation")
    except:
        print("Unable to Replace XLAFLashAttenWrapper With Original Attention Implementation")
    model = model.cpu()



In [ ]:
## model.push_to_hub()
## tokenizer.push_to_hub()